In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen
import re
import time
import requests

In [2]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page='
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [3]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [4]:
total_num_of_items = 0
time_stamp = time.strftime("%d/%m/%Y %H:%M:%S")

In [5]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [7]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [9]:
len(containers)

12

In [11]:
containers[0]

<div class="js-react-proj-card grid-col-12 grid-col-6-sm grid-col-4-lg" data-pid="786901944" data-project='{"id":786901944,"photo":{"key":"assets/036/492/163/7000743662c834d73f897063b290a515_original.jpeg","full":"https://ksr-ugc.imgix.net/assets/036/492/163/7000743662c834d73f897063b290a515_original.jpeg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=560&amp;h=315&amp;fit=crop&amp;v=1646031479&amp;auto=format&amp;frame=1&amp;q=92&amp;s=da0655279d90bc4c4603392c59f52bfe","ed":"https://ksr-ugc.imgix.net/assets/036/492/163/7000743662c834d73f897063b290a515_original.jpeg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=352&amp;h=198&amp;fit=crop&amp;v=1646031479&amp;auto=format&amp;frame=1&amp;q=92&amp;s=d4279c031741a190214fa92bb58f3f72","med":"https://ksr-ugc.imgix.net/assets/036/492/163/7000743662c834d73f897063b290a515_original.jpeg?ixlib=rb-4.0.2&amp;crop=faces&amp;w=272&amp;h=153&amp;fit=crop&amp;v=1646031479&amp;auto=format&amp;frame=1&amp;q=92&amp;s=449612fed77418290b686af14af043b3","little":"https://ksr-ugc.

In [19]:
whole_string = str(containers[1]).replace('&amp', '').replace('&quot;', '')

In [20]:
if '"name"' in whole_string:
    project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
else:
    project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]    

In [21]:
project_info_dict = scraper_prototype(project_info, 'project')
df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()

In [22]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,project_launched_at,project_staff_pick,project_is_starrable,project_backers_count,project_static_usd_rate,project_usd_pledged,project_converted_pledged_amount,project_fx_rate,project_usd_exchange_rate,project_current_currency
0,The most innovative Hair Towels EVER MADE for ...,A selection of large microfiber hair towel tur...,14000.0,14721.32,live,the-most-innovative-hair-towels-ever-made-for-...,false,AU,Australia,AUD,...,1643658704,false,false,127,0.69869972,10285.7821620304,10699,0.72681021,0.72681021,USD


In [23]:
df1

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,project_launched_at,project_staff_pick,project_is_starrable,project_backers_count,project_static_usd_rate,project_usd_pledged,project_converted_pledged_amount,project_fx_rate,project_usd_exchange_rate,project_current_currency
0,Octagon Rainbow -- Fountain Pens in dream colors,Beautifully crafted fountain pens of rainbow w...,3000.0,16294.29,live,octagon-rainbow-fountain-pens-in-dream-colors,false,CA,Canada,CAD,...,1643832012,false,false,121,0.78744932,12830.9275803828,12809,0.78615116,0.78615116,USD


In [24]:
if '"creator"' in whole_string:
    creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
else:
    creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]

In [26]:
creator_info_dict = scraper_prototype(creator_string, 'creator')
creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()

In [27]:
df2

,creator_id,creator_name,creator_slug,creator_is_registered,creator_is_email_verified,creator_chosen_currency,creator_is_superbacker,creator_avatar,creator_small
0,304407774,Artimber,artimber,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/033/46...,https://ksr-ugc.imgix.net/assets/033/467/402/d...


In [30]:
if '"location"' in whole_string:
    location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
else:
    location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]

In [31]:
location_info_dict = scraper_prototype(location_string, 'location')
df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()

In [32]:
df3

,location_id,location_name,location_slug,location_short_name,location_displayable_name,location_localized_name,location_country,location_state,location_type,location_is_root,location_expanded_country,location_urls
0,2151849,Shanghai,shanghai-cn,"Shanghai, China","Shanghai, China",Shanghai,CN,Shanghai,Town,false,China,{web:{discover:https://www.kickstarter.com/dis...


In [33]:
if '"profile"' in whole_string:
    profile_string = re.findall(r'\"profile\"\:\{(.+?)\}', whole_string)[0]
else:
    profile_string = re.findall(r'profile\:\{(.+?)\}', whole_string)[0]

In [34]:
profile_info_dict = scraper_prototype(profile_string, 'profile')
df4 = pd.DataFrame.from_dict(profile_info_dict, orient='index').transpose()

In [35]:
df4

,profile_id,profile_project_id,profile_state,profile_state_changed_at,profile_name,profile_blurb,profile_background_color,profile_text_color,profile_link_background_color,profile_link_text_color,profile_link_text,profile_link_url,profile_show_feature_image,profile_background_image_opacity,profile_should_show_feature_image_section,profile_feature_image_attributes
0,4314001,4314001,inactive,1637722607,null,null,null,null,null,null,null,null,false,0.8,true,{image_urls:{default:https://ksr-ugc.imgix.net...


In [36]:
df = pd.concat((df1, df2, df3, df4), axis=1)

In [37]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,profile_background_color,profile_text_color,profile_link_background_color,profile_link_text_color,profile_link_text,profile_link_url,profile_show_feature_image,profile_background_image_opacity,profile_should_show_feature_image_section,profile_feature_image_attributes
0,Octagon Rainbow -- Fountain Pens in dream colors,Beautifully crafted fountain pens of rainbow w...,3000.0,16294.29,live,octagon-rainbow-fountain-pens-in-dream-colors,false,CA,Canada,CAD,...,null,null,null,null,null,null,false,0.8,true,{image_urls:{default:https://ksr-ugc.imgix.net...
